In [ ]:
# 可导入任何角色，初始化它，用一个开始的消息运行它，完成！
from metagpt.roles.product_manager import ProductManager
prompt = f"""
# Role：软件开发团队

## Background :

我是一个软件开发团队。
现在要用html、js、css、flask开发一个刷题程序。
刷题可以让人们对题目中涉及的知识点有更深的掌握。

## Profile:
- author: 黎伟
- version: 0.1
- language: 中文
- description: 我是一软件开发团队。

## Goals:
- 用html、js、css、flask开发一个刷题程序。

## Constrains:
1. 最后交付的程序是一个html单文件，不要有其他任何文件。
2. 题目的题型至少包括判断题、选择题、填空题。
3. 题目的内容与人工智能的agent基本理论相关。
4. 刷题程序至少给出10道样例题目。
5. 题目用列表的形式写到html文件的script部分。
6. 后端使用flask。
7. 不用数据库，pandas后台存储和处理数据，存储了10个判断题、10道选择题、2道填空题。
8. 每次答题随机出3道题目。
9. 用户可以提交他的答题情况，并对答题情况进行判断是否正确。

## Skills:
1. 具有强大的js语言开发能力
2. 熟悉css、html、pandas的使用
3. 对人工智能的agent基本理论有较好理解
4. 拥有排版审美, 会利用序号, 缩进, 分隔线和换行符等等来美化信息排版
5. 界面酷炫,蓝色和黄色搭配色调,操作要有过渡动画和科幻效果


请结合上述要求完善刷题程序的开发需求文档，并完成开发。

"""
# async def main():
#     role = ProductManager()
#     result = await role.run(prompt)
#     # print(result)
    
# await main()

In [ ]:
import asyncio
from metagpt.roles import (
    Architect,
    Engineer,
    ProductManager,
    ProjectManager,
    QaEngineer,
)
from metagpt.team import Team
async def startup(idea: str):
    company = Team()
    company.hire(
        [
            ProductManager(),
            Architect(),
            ProjectManager(),
            Engineer(),
            QaEngineer()

        ]
    )
    company.invest(investment=1000.0)
    company.run_project(idea=idea)

    await company.run(n_round=1000)

await startup(idea=prompt)

In [3]:
prompt = f"""
# Role：软件开发团队

## Background :

我是一个软件开发团队。
现在要用html、js、css、flask开发一个即使通讯软件。
刷题可以让人们对题目中涉及的知识点有更深的掌握。

## Profile:
- author: 黎伟
- version: 0.1
- language: 中文
- description: 我是一软件开发团队。

## Goals:
- 用html、js、css、flask开发一个刷题程序。

## Constrains:
1. 最后交付的程序是一个html单文件，不要有其他任何文件。
2. 类似钉钉的样子。
3. 可以私聊、群聊。
4. 采用左中右布局，左侧是功能菜单，中间是左侧点击后的功能页面或者是聊天组队列，右侧是中间点击具体聊天组的聊天界面。
5. 用列表的形式写到html文件的script部分。
6. 后端使用flask。
7. 不用数据库，pandas后台存储和处理数据和转发消息。


## Skills:
1. 具有强大的js语言开发能力
2. 熟悉css、html、pandas的使用
3. 对即使通讯软件的开发有较好理解
4. 拥有排版审美, 会利用序号, 缩进, 分隔线和换行符等等来美化信息排版
5. 界面酷炫,蓝色和黄色搭配色调,操作要有过渡动画和科幻效果


请结合上述要求完善刷题程序的开发需求文档，并完成开发。

"""

In [ ]:

await startup(idea=prompt)

In [1]:
from metagpt.actions import Action

class SimpleWriteCode(Action):
    PROMPT_TEMPLATE: str = """
    Write a python function that can {instruction} and provide two runnnable test cases.
    Return ```python your_code_here ```with NO other texts,
    your code:
    """

    name: str = "SimpleWriteCode"

    async def run(self, instruction: str):
        prompt = self.PROMPT_TEMPLATE.format(instruction=instruction)

        rsp = await self._aask(prompt)

        code_text = SimpleWriteCode.parse_code(rsp)

        return code_text

    @staticmethod
    def parse_code(rsp):
        pattern = r"```python(.*)```"
        match = re.search(pattern, rsp, re.DOTALL)
        code_text = match.group(1) if match else rsp
        return code_text

2025-01-23 15:43:17.350 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to e:\wow-agent\notebook


In [2]:
import re
import os
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.logs import logger

class SimpleCoder(Role):
    name: str = "Alice"
    profile: str = "SimpleCoder"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([SimpleWriteCode])

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo  # todo will be SimpleWriteCode()

        msg = self.get_memories(k=1)[0]  # find the most recent messages
        code_text = await todo.run(msg.content)
        msg = Message(content=code_text, role=self.profile, cause_by=type(todo))

        return msg

In [3]:
async def main():
    msg = "write a function that calculates the sum of a list"
    role = SimpleCoder()
    logger.info(msg)
    result = await role.run(msg)
    logger.info(result)
    return result
    
        
rtn = await main()

2025-01-23 15:43:26.473 | INFO     | __main__:main:4 - write a function that calculates the sum of a list
2025-01-23 15:43:26.475 | INFO     | __main__:_act:16 - Alice(SimpleCoder): to do SimpleWriteCode(SimpleWriteCode)


```python
def sum_list(numbers):
    return sum(numbers)

# Test case 1
assert sum_list([1, 2, 3, 4, 5]) == 15, "Test case 1 failed"

# Test case 2
assert sum_list([-1, -2, -3, -4, -5]) == -15, "Test case 2 failed"

print("All test cases passed!")
```


2025-01-23 15:43:28.242 | WARNING  | metagpt.provider.openai_api:_calc_usage:244 - usage calculation failed: num_tokens_from_messages() is not implemented for model Qwen2.5-32B-Instruct-AWQ. See https://cookbook.openai.com/examples/how_to_count_tokens_with_tiktoken for information on how messages are converted to tokens.
2025-01-23 15:43:28.243 | INFO     | __main__:main:6 - SimpleCoder: 
def sum_list(numbers):
    return sum(numbers)

# Test case 1
assert sum_list([1, 2, 3, 4, 5]) == 15, "Test case 1 failed"

# Test case 2
assert sum_list([-1, -2, -3, -4, -5]) == -15, "Test case 2 failed"

print("All test cases passed!")



In [4]:
def sum_list(numbers):
    return sum(numbers)

# Test case 1
assert sum_list([1, 2, 3, 4, 5]) == 15, "Test case 1 failed"

# Test case 2
assert sum_list([-1, -2, -3, -4, -5]) == -15, "Test case 2 failed"

print("All test cases passed!")

All test cases passed!


In [5]:
def sum_list(numbers):
    return sum(numbers)

# Test case 1
assert sum_list([1, 2, 3, 4, 5]) == 15, "Test case 1 failed"

# Test case 2
assert sum_list([-1, -2, -3, -4, -5]) == -15, "Test case 2 failed"

print("All test cases passed!")

All test cases passed!
